In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy 
import pandas as pd
import pickle as pkl
import dataframe

In [14]:
'''df = pd.read_csv('data/tweets_ats.csv')
tweets = df['renderedContent'].tolist()'''

In [2]:
app_store = pkl.load(open("./data/app_store.pkl", "rb"))
app_store = pd.json_normalize(app_store)

play_store = pkl.load(open("./data/play_store.pkl", "rb"))
play_store = pd.json_normalize(play_store)
df = dataframe.GetDataframeStores(play_store,app_store)

c:\Users\yanni\OneDrive\FHNW\Challenges\MAKEathon\dataframe.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_app["content"] = df_app["title"] + ": " + df_app["review"]


In [17]:
df.reset_index()

,index,rating,content,language_prob
0,0,3,Übersicht: Bitte wieder konto gesamt übersicht...,de
1,1,2,Super verschlimmbessert: Waren gerade alle UI ...,de
2,2,1,Ein Rückschritt: Die alte Postfinanceapp (vor ...,de
3,3,1,"Gute Funktionen verschwunden: Saldoverlauf, Li...",de
4,4,1,Bitte gebt uns die Gesamtansicht der Konten zu...,de
...,...,...,...,...
9925,7265,5,"Funktioniert einwandfrei, auch der Scan von Ei...",de
9926,7266,5,super. sehr gute mobile banking App,id
9927,7267,5,"Top App, danke",af
9928,7268,4,Endlich ist es da.,de


In [21]:
df.reset_index()
df = df[df['language_prob'] == 'de']
tweets = df['content'].tolist()

In [30]:
zeichen = []

for tweet in tweets:
    if len(tweet) > 514:
        zeichen.append(len(tweet))
    print(zeichen.count())

TypeError: list.count() takes exactly one argument (0 given)

In [4]:
# load model and tokenizer in English
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [23]:
# Clustering the Sentences
positive = []
negative = []
neutral = []

count = 0

for tweet in tweets:
    count += 1
    print(count)
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
    
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    positive.append(scores[0])
    negative.append(scores[1])
    neutral.append(scores[2])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


RuntimeError: The expanded size of the tensor (517) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 517].  Tensor sizes: [1, 514]